In [355]:

import oauth2client
from oauth2client.client import GoogleCredentials
from oauth2client import client
from oauth2client.client import flow_from_clientsecrets
from oauth2client import file
from oauth2client import tools
from googleapiclient import discovery
from googleapiclient import errors
from google_auth_oauthlib import flow
import google_auth_httplib2
import httplib2
import urllib.request
import time
import os
import io
from io import StringIO
import numpy as np
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
import json

In [356]:
OAUTH_SCOPES = ("https://www.googleapis.com/auth/dfareporting " + "https://www.googleapis.com/auth/dfatrafficking " + "https://www.googleapis.com/auth/ddmconversions ")

In [357]:
# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    "/Users/hamza.ahmed/Coding Projects/Python Apps Authentications/Campaign Manager/client_secret.json", scope = OAUTH_SCOPES, 
    redirect_uri = "urn:ietf:wg:oauth:2.0:oob")

storage = oauth2client.file.Storage('/Users/hamza.ahmed/Coding Projects/Python Apps Authentications/Campaign Manager/credentials.dat')
credentials = storage.get()

if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage,
                               tools.argparser.parse_known_args()[0])
http = credentials.authorize(httplib2.Http())


service = discovery.build('dfareporting', 'v3.5', http=http)

In [358]:
request = service.userProfiles().list()
response = request.execute()

for profile in response['items']:
  print('Found user profile with ID %s and user name "%s".' %
        (profile['profileId'], profile['userName']))

Found user profile with ID 7323797 and user name "hamza.ahmed".


In [359]:
AllReportIds = []
report_names_ = []

# Construct the request.
request = service.reports().list(profileId= 7323797, scope = "ALL")

response = request.execute()

for report in response['items']:
  AllReportIds.append(report['id'])
  report_names_.append(report['name'])

In [360]:
AllReportIds

['932474551',
 '931935343',
 '840586381',
 '870632981',
 '905182599',
 '930116908',
 '872963562',
 '872963704',
 '927501164',
 '924658802']

In [361]:
  # report_file = service.files().get(
  #     reportId=870632981, fileId=3629104741).execute()
  # report_file

In [362]:
#  def tokenize(str):
#     idx = [x for x, v in enumerate(str) if v == '\"']
#     if len(idx) % 2 != 0:
#         idx = idx[:-1]
#     memory = {}
#     for i in range(0, len(idx), 2):
#         val = str[idx[i]:idx[i+1]+1]
#         key = "_"*(len(val)-1)+"{0}".format(i)
#         memory[key] = val
#         str = str.replace(memory[key], key, 1)        
#     return [memory.get(token, token) for token in str.split(",")]  

In [363]:
# report_file['id']

In [364]:
file_ids = []
for ids in AllReportIds:
    report_file = service.reports().run(
        profileId=7323797, reportId=ids).execute()
    file_ids.append(report_file['id'])

In [365]:
file_ids

['3636654011',
 '3636654012',
 '3636654013',
 '3636653946',
 '3636653950',
 '3636654024',
 '3636653961',
 '3636653964',
 '3636654032',
 '3636654101']

In [366]:
report_names_

['Untitled Report',
 'Datorama Delivery Report: 3632067',
 'Varicent 2022',
 'JumpCloud',
 'Datorama Delivery Report: 3632067',
 'Datorama Delivery Report: 3632067',
 'Adjuster_Report_Do_Not_Modify_19957',
 'Adjuster_Report_Do_Not_Modify_19957',
 'Datorama Delivery Report: 3632067',
 'Untitled Report']

In [368]:
len(report_names_)

10

In [325]:
# Prepare a local file to download the report contents to.

# for names,reportids in zip(report_names_, AllReportIds): 
#     out_file = io.FileIO(f'{names}_{reportids}', mode='wb')

In [326]:
request = service.files().get_media(reportId=932474551, fileId=3635928262).execute()

In [369]:
list(zip(file_ids, AllReportIds))

[('3636654011', '932474551'),
 ('3636654012', '931935343'),
 ('3636654013', '840586381'),
 ('3636653946', '870632981'),
 ('3636653950', '905182599'),
 ('3636654024', '930116908'),
 ('3636653961', '872963562'),
 ('3636653964', '872963704'),
 ('3636654032', '927501164'),
 ('3636654101', '924658802')]

In [372]:
All_requests_files = []
for fileids, reportids in list(zip(file_ids, AllReportIds)):
    request = service.files().get_media(reportId=reportids, fileId=fileids)
    if request is not None:
        try:
            r = request.execute()
        except errors.HttpError:
            print('Not found')
    All_requests_files.append(r)



Not found


In [373]:
len(All_requests_files)

10

In [375]:
datum = []
dataframes = []
for request_files in All_requests_files:  
    result = str(request_files, 'utf-8')  
    data = StringIO(result)
    datum.append(data)
    lines = list(map(lambda x: tokenize(x.strip()), data))
    df = pd.DataFrame(lines).replace(np.nan, '')
    dataframes.append(df)

In [376]:
import os

folderpath = "/Users/hamza.ahmed/Coding Projects/Data/Campaign Manager Data"
csv = 'csv'  # output file type
for i, df in enumerate(dataframes, 1):
    filename = "df_{}.{}".format(i, csv)
    filepath = os.path.join(folderpath, filename)
    df.to_csv(filepath)

In [356]:
import numpy as np
s=str(request,'utf-8')

data = StringIO(s) 

# df=pd.read_csv(data, delimiter="/n", header=None)
df.to_csv('out.csv', encoding='utf-8')
lines = list(map(lambda x: tokenize(x.strip()), data))
df = pd.DataFrame(lines).replace(np.nan, '')
df.to_csv('out.csv', encoding='utf-8')

In [352]:
# import os

# folderpath = "/Users/hamza.ahmed/Coding Projects/Data/Campaign Manager Data"
# def list_files(path):
#     # returns a list of names (with extension, without full path) of all files 
#     # in folder path
#     files = []
#     for name in os.listdir(path):
#         if os.path.isfile(os.path.join(path, name)):
#             files.append(name)
#     return files 

# All_Files = list_files(folderpath)


In [353]:
# All_Files

['Adjuster_Report_Do_Not_Modify_19957.csv',
 'JumpCloud.csv',
 'Datorama Delivery Report: 3632067.csv',
 'Varicent 2022.csv',
 'Untitled Report.csv']

In [350]:
# for names, i in zip(report_names_, All_Files):
#     os.rename(os.path.join(folderpath, i), os.path.join(folderpath,names +'.csv'))

In [284]:
# # Retrieve the file metadata.
# for reportids, fileids in list(zip(AllReportIds, file_ids)):
#       request = service.files().get_media(reportId=reportids, fileId=fileids)
#       downloader =MediaIoBaseDownload(out_file, request)
#       done = False
#       while done is False:
#             status, done = downloader.next_chunk()
#       break 

      

In [285]:
# for x in range(0,10):
#     print(All_report_files[x]['status'])


In [286]:
# downloader =MediaIoBaseDownload(
#       out_file, request)
#          # Execute the get request and download the file.
#       download_finished = False
#       while download_finished is False:
#          _, download_finished = downloader.next_chunk()

#       print('File %s downloaded to %s' % (fileids,
#                                           os.path.realpath(out_file.name)))


In [287]:
# request = service.files().get_media(reportId=reportids, fileId=fileids)
#       downloader =MediaIoBaseDownload(
#       out_file, request)
#          # Execute the get request and download the file.
#       download_finished = False
#       while download_finished is False:
#          _, download_finished = downloader.next_chunk()

#       print('File %s downloaded to %s' % (reportids,
#                                           os.path.realpath(out_file.name)))

In [288]:
# # # Retrieve the file metadata.
# # report_file = service.files().get(
# #     reportId=870632981, fileId=3630090510).execute()

# # if report_file['status'] == 'REPORT_AVAILABLE':
# #   # Prepare a local file to download the report contents to.
# #   out_file = io.FileIO("poop.csv", mode='wb')

#   # Create a get request.
#   request = service.files().get_media(reportId=870632981, fileId=3630090510)

#   # Create a media downloader instance.
#   # Optional: adjust the chunk size used when downloading the file.
#   downloader =MediaIoBaseDownload(
#       out_file, request)

#   # Execute the get request and download the file.
#   download_finished = False
#   while download_finished is False:
#     _, download_finished = downloader.next_chunk()

#   print('File %s downloaded to %s' % (report_file['id'],
#                                       os.path.realpath(out_file.name)))

In [293]:


# if report_file['status'] == 'REPORT_AVAILABLE':
#   # Prepare a local file to download the report contents to.
#   out_file = open("myfile.txt", "x")

#   # Create a get request.
#   request = service.files().get_media(reportId=921909340, fileId=3630090255)

#   # Create a media downloader instance.
#   # Optional: adjust the chunk size used when downloading the file.
#   downloader = http.MediaIoBaseDownload(
#       out_file, request, chunksize=CHUNK_SIZE)

#   # Execute the get request and download the file.
#   download_finished = False
#   while download_finished is False:
#     _, download_finished = downloader.next_chunk()

#   print('File %s downloaded to %s' % (report_file['id'],
#                                       os.path.realpath(out_file.name)))